# Scrape Seeking Alpha

In [18]:
# Jupyter magic
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# imports
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import pandas as pd
from seleniumrequests import Chrome
import pathlib

In [20]:
save_dir = "../1_data/Seeking_Alpha"
# pathlib.Path(save_dir).mkdir()
pathlib.Path(save_dir).exists()

True

## Create index of URLs for earnings call transcripts

In [10]:
# Get index of article links and other metadata
list_item_df = pd.DataFrame()

for i in range(1,2):
    # Just all transcripts
#     url = "https://seekingalpha.com/earnings/earnings-call-transcripts/" + str(i)

    # Search for keyword
    url = 'https://seekingalpha.com/search?q=innovation&type=keyword&tab=transcripts#page=' + str(i)
    print(url)
    response = requests.get(url)
    print(response)
    soup = BeautifulSoup(response.text, 'html.parser')
    listitems= soup.findAll("li", {"class": "list-group-item article"})
    
    for listitem in listitems:
        link = listitem.find("a", {"class": "dashboard-article-link"})
        title = link.text
        article_url = link.get('href')

        desc = listitem.find('div')
        org = desc.find('span').find('a').get('title')
        date = re.search('•\n      (.*)\n\n        •', desc.text).group(1)
        # Add metadata to index
        list_item_df = list_item_df.append({
            'title': title
            , 'url': 'https://seekingalpha.com' + article_url
            , 'org': org
            , 'date': date
        }, ignore_index=True)
    time.sleep(random.random())

https://seekingalpha.com/search?q=innovation&type=keyword&tab=transcripts#page=1
<Response [403]>


In [11]:
list_item_df

""


In [12]:
listitem = listitems[1]

IndexError: list index out of range

## Setup the Selenium Window for Querying

This part requries downloading a driver for selenium. I used Chrome to get this running.


In [16]:
!pwd

/Users/ahale/Projects/a-list/notebooks


In [33]:
# Change this to where the chromedriver is in your environment
path_to_chromedriver = '/Users/ahale/chromedriver'

In [42]:
url = "https://seekingalpha.com/earnings/earnings-call-transcripts/" + '2000'
driver = Chrome(executable_path=path_to_chromedriver)
driver.get(url)

In [ ]:
list_item_df = pd.DataFrame()

In [45]:
# Get index of all articles
for i in range(1,100):
    url = "https://seekingalpha.com/earnings/earnings-call-transcripts/" + str(i)
    display(url)
    driver = Chrome(executable_path=path_to_chromedriver)
    driver.get(url)
    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    listitems= soup.findAll("li", {"class": "list-group-item article"})

    for listitem in listitems:
        link = listitem.find("a", {"class": "dashboard-article-link"})
        title = link.text
        article_url = link.get('href')

        desc = listitem.find('div')
        org = desc.find('span').find('a').get('title')
        date = re.search('•\n      (.*)\n\n        •', desc.text).group(1)
        # Add metadata to index
        list_item_df = list_item_df.append({
            'title': title
            , 'url': 'https://seekingalpha.com' + article_url
            , 'org': org
            , 'date': date
        }, ignore_index=True)
    time.sleep(random.random())
    
    # Close the chrome window so I don't end up with 1000s open on my machine
    driver.close()
list_item_df

'https://seekingalpha.com/earnings/earnings-call-transcripts/1'

,date,org,title,url
0,"Fri, Nov. 20, 3:35 PM",PennantPark Investment,PennantPark Investment Corporation (PNNT) CEO ...,https://seekingalpha.com/article/4390727-penna...
1,"Fri, Nov. 20, 3:31 PM",Geospace Technologies Corporation,Geospace Technologies Corporation (GEOS) CEO W...,https://seekingalpha.com/article/4390719-geosp...
2,"Fri, Nov. 20, 3:25 PM",The RMR Group Inc.,The RMR Group's (RMR) CEO Adam Portnoy on Q4 2...,https://seekingalpha.com/article/4390712-rmr-g...
3,"Fri, Nov. 20, 3:02 PM",Sysco Corporation,Sysco Corporation (SYY) 2020 Annual Meeting of...,https://seekingalpha.com/article/4390698-sysco...
4,"Fri, Nov. 20, 3:01 PM",Real Matters Inc.,Real Matters' (RLLMF) CEO Brian Lang on Q4 202...,https://seekingalpha.com/article/4390697-real-...
...,...,...,...,...
107,"Thu, Nov. 19, 11:31 PM",Halma plc,Halma's (HLMAF) CEO Andrew Williams on Half Ye...,https://seekingalpha.com/article/4390518-halma...
108,"Thu, Nov. 19, 11:28 PM","Beacon Roofing Supply, Inc.","Beacon Roofing Supply, Inc. (BECN) CEO Julian ...",https://seekingalpha.com/article/4390516-beaco...
109,"Thu, Nov. 19, 11:26 PM","Williams-Sonoma, Inc.","Williams-Sonoma, Inc. (WSM) CEO Laura Alber on...",https://seekingalpha.com/article/4390515-willi...
110,"Thu, Nov. 19, 11:25 PM","Delta Apparel, Inc.","Delta Apparel, Inc. (DLA) CEO Robert Humphreys...",https://seekingalpha.com/article/4390514-delta...


In [47]:
list_item_df.to_csv('../1_data/Seeking_Alpha/index.csv')

In [46]:
list_item_df.url[0]

'https://seekingalpha.com/article/4390727-pennantpark-investment-corporation-pnnt-ceo-art-penn-on-q4-2020-results-earnings-call'

In [31]:
links = driver.find_elements_by_class_name('list-group-item article')
len(links)

0

## pull the data and write a csv

In [ ]:
post_data = now_query.make_first_post_data(search = search)
initial_query = now_query.make_first_query(search)

full_data = []
all_pages = []
response = now_query.query_request(driver, initial_query)
print('response', response)
pages, df = now_query.parse_query_response(response)
full_data.append(df)
all_pages.append(pages)

total_pages = pages['last_page']
print(f'total_pages: {total_pages}')
last_query = initial_query


max_pages = 15

if total_pages is None:
    total_pages = 1

if total_pages > max_pages:
    print(f"instead of the full {total_pages}, we are only scraping {max_pages}")
    total_pages = max_pages

for i in range(total_pages - 1):
    

    this_query = "/now/" + pages['next_ref']
    print('this:', this_query)
    print('last:', last_query)

    delay = 4 + random.random() * 6
    time.sleep(delay)
    response = now_query.query_request(driver, this_query, last_query)
    pages, df = now_query.parse_query_response(response)

    full_data.append(df)
    all_pages.append(pages)
    
    last_query = this_query

    print(len(df), 'data rows pulled')
    print(len(full_data))
    

full_df = pd.concat(full_data, axis=0)
full_df.to_csv(csv_name)

## Check if it worked

In [ ]:
full_df.describe().T

In [ ]:
pd.read_csv(csv_name)

In [18]:
url = "https://www.ideo.com/journal/archive/2"

response = requests.get(url)
print(response)
soup = BeautifulSoup(response.text, 'html.parser')
links2 = [a.get('href') for a in soup.find_all('a')]
len(links2)

<Response [200]>


109

In [19]:
unique_links = links1 + links2
print(f'before dedupe: {len(unique_links)}')
unique_links = list(set(unique_links))
len(unique_links)

before dedupe: 188


72

In [46]:
article_links = [l for l in unique_links if '/journal/' in l]
article_links.remove('/journal/archive')

full_urls = ['https://www.ideo.com' + link for link in article_links]


len(full_urls)

35

In [47]:
# Save them as backup
with open(f'{save_dir}/index.txt', 'w') as f:
    f.writelines(full_urls)

## Grab the text for each post

In [50]:
full_urls[0]

'https://www.ideo.com/journal/the-best-business-breakthroughs-come-from-moments-of-doubt'

In [49]:
article_url = full_urls[0]
response = requests.get(full_url)
print(response)
soup = BeautifulSoup(response.text, 'html.parser')


<Response [200]>


In [45]:
full_url

'https://www.ideo.com/journal/the-best-business-breakthroughs-come-from-moments-of-doubt'

In [42]:
title = soup.find('h1').text


'The Best Business Breakthroughs Come From Moments of Doubt'

In [60]:
junk = [
'Send this to a friend:',
 'Related Posts',
 'Send this to a friend:',
 'Say Hello.',
 'Come work with us! We are always looking for great talent to join our global teams.',
'Send this to a friend:',
]

def get_article(article_url):
    response = requests.get(article_url)
    print(response)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('h1').text
    text = [t.text for t in soup.find_all(['p', 'h2'])]
    
    for j in junk:
        text.remove(j)
    
    

    
    return title, text

title, text = get_article(full_url)


<Response [200]>


In [62]:
for article_url in full_urls:
    title, text = get_article(article_url)

    print(title)
    print(len(text), 'lines')
    
    fname = f"{save_dir}/{title}.txt"
    print(fname)
    with open(fname, 'w') as f:
        f.writelines(text)

<Response [200]>
The Best Business Breakthroughs Come From Moments of Doubt
34 lines
../1_data/IDEO_journal/The Best Business Breakthroughs Come From Moments of Doubt.txt'
<Response [200]>
5 Leaders Reflect on How the Pandemic Has Reshaped Their Thinking
32 lines
../1_data/IDEO_journal/5 Leaders Reflect on How the Pandemic Has Reshaped Their Thinking.txt'
<Response [200]>
Innovation Thrives Under These 5 Evidence-Backed Conditions
16 lines
../1_data/IDEO_journal/Innovation Thrives Under These 5 Evidence-Backed Conditions.txt'
<Response [200]>
Prototype the Future of Your Business With This 4-Step Design Exercise
27 lines
../1_data/IDEO_journal/Prototype the Future of Your Business With This 4-Step Design Exercise.txt'
<Response [200]>
Company Say, Company Do: How Your Words Define Your Business
5 lines
../1_data/IDEO_journal/Company Say, Company Do: How Your Words Define Your Business.txt'
<Response [200]>
Resilience in Uncertain Times: Insights From Global Leaders
23 lines
../1_data/I

In [66]:
# for n in pathlib.Path(save_dir).glob('*'):
#     n.rename(n.name.strip("'"))

In [ ]:
from seleniumrequests import Chrome